# **Examen Bimestral – Diseño de un Sistema Básico de Recuperación de Información**
**Nombre: Dilan Andrade**

Construir un sistema de recuperación de información que sea capaz de
responder consultas textuales

### Explicación de Librerías Importadas

Este proyecto utiliza las siguientes bibliotecas para el análisis de datos y procesamiento de lenguaje natural (NLP):

- **Pandas**: Manipulación y análisis de datos en formato de tablas (DataFrames).
- **OS**: Interacción con el sistema operativo (gestión de archivos y directorios).
- **spaCy**: Procesamiento de texto, como tokenización y lematización.
- **re**: Expresiones regulares para manipulación de texto.
- **NumPy**: Cálculos matemáticos y manejo de arreglos.
- **sklearn**:  
  - **CountVectorizer** y **TfidfVectorizer**: Conversión de texto en matrices de frecuencia y ponderación.
  - **cosine_similarity**: Medición de similitud entre vectores de texto.
- **Gensim**: Modelado de temas y representaciones vectoriales de palabras (Word2Vec).
- **NLTK**: Herramientas de NLP, como tokenización, lematización y etiquetado gramatical.
  - **stopwords**: Palabras vacías a eliminar en el preprocesamiento.
  - **wordnet**: Base de datos léxica de sinónimos.
  - **WordNetLemmatizer**: Lematización de palabras.
  - **word_tokenize**: Tokenización de texto en palabras.
  - **pos_tag**: Etiquetado de partes del habla.
- **collections**:  
  - **defaultdict**: Diccionario con valor predeterminado para claves no existentes.


In [15]:
import pandas as pd
import os
import spacy
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import defaultdict

### Descargas de NLTK

- **stopwords**: Palabras vacías que se eliminan en el preprocesamiento de texto.
- **punkt**: Herramienta para tokenización (dividir texto en palabras).
- **averaged_perceptron_tagger_eng**: Etiquetado gramatical de palabras (por ejemplo, sustantivos, verbos).
- **wordnet**: Base de datos de sinónimos y relaciones semánticas.

In [ ]:
# Descargar recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

# Procesamiento y combinación información:
1. **Carga y Selección de Datos**:  
   Se cargan los archivos rotten_tomatoes_movies.csv y rotten_tomatoes_critic_reviews.csv, seleccionando solo las columnas relevantes.
2. **Unión de DataFrames**:  
   Se realiza una unión interna usando rotten_tomatoes_link como clave común entre ambos DataFrames.
3. **Limpieza y Agrupación**:  
   Se reemplazan los valores NaN en las críticas con cadenas vacías y se agrupan las críticas por película, concatenándolas en una sola celda separada por comas.
4. **Guardar y Visualizar**:  
   El DataFrame resultante se guarda en un nuevo archivo CSV, merged_movies_reviews.csv, y se muestran las primeras filas del resultado.

In [17]:
# Cargar los archivos CSV
movies_df = pd.read_csv('rotten_tomatoes_movies.csv')  # Archivo con información de películas
reviews_df = pd.read_csv('rotten_tomatoes_critic_reviews.csv')  # Archivo con críticas

# Seleccionar solo las columnas necesarias de cada archivo
movies_df = movies_df[['rotten_tomatoes_link', 'movie_title', 'movie_info', 'genres', 'tomatometer_status']]
reviews_df = reviews_df[['rotten_tomatoes_link', 'review_content']]

# Realizar la unión de los DataFrames usando la columna 'rotten_tomatoes_link'
merged_df = pd.merge(movies_df, reviews_df, on='rotten_tomatoes_link', how='inner')

# Asegurarse de que las críticas sean cadenas de texto, reemplazando los valores NaN por cadenas vacías
merged_df['review_content'] = merged_df['review_content'].fillna('')

# Agrupar las críticas por 'rotten_tomatoes_link' y concatenarlas en una sola celda
merged_df = merged_df.groupby(
    ['rotten_tomatoes_link', 'movie_title', 'movie_info', 'genres', 'tomatometer_status'],
    as_index=False
).agg({'review_content': lambda x: ', '.join(x)})

# Guardar el resultado en un nuevo archivo CSV
merged_df.to_csv('merged_movies_reviews.csv', index=False)

# Mostrar las primeras filas del DataFrame resultante
print(merged_df.head())

                    rotten_tomatoes_link  \
0                              m/0814255   
1                              m/0878835   
2                                   m/10   
3                 m/1000013-12_angry_men   
4  m/1000079-20000_leagues_under_the_sea   

                                         movie_title  \
0  Percy Jackson & the Olympians: The Lightning T...   
1                                        Please Give   
2                                                 10   
3                    12 Angry Men (Twelve Angry Men)   
4                       20,000 Leagues Under The Sea   

                                          movie_info  \
0  Always trouble-prone, the life of teenager Per...   
1  Kate (Catherine Keener) and her husband Alex (...   
2  A successful, middle-aged Hollywood songwriter...   
3  Following the closing arguments in a murder tr...   
4  In 1866, Professor Pierre M. Aronnax (Paul Luk...   

                                              genres tomatome

# Preprocesamiento:

* **stopwords_set**:  
   Conjunto de palabras vacías (stopwords) en inglés, obtenidas de NLTK, que se utilizan para filtrar palabras no relevantes en el análisis de texto.

* **lemmatizer**:  
   Instancia del lematizador WordNetLemmatizer de NLTK, que se utiliza para reducir las palabras a su forma raíz.

* **signos_puntuacion**:  
   Lista de signos de puntuación y caracteres especiales que se pueden utilizar para eliminar o manejar durante el procesamiento del texto.


In [18]:
# Variables de configuración
stopwords_set = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
signos_puntuacion = [",", ".", ":", "...", "-", "_", "+", ";", '"', "(", ")", "[", "]", "%", "$", "#", "@", "&", "?", "!", "/", ">", "<"]

**get_wordnet_pos**:
   Esta función convierte las etiquetas  (como sustantivos, verbos, adjetivos, etc.) que NLTK utilizara. Con esto el lematizador entiende qué tipo de palabra estamos procesando para que pueda reducirla a su forma base, como convertir "corriendo" en "correr".

**preprocesar_contenido**:
   - **Convertir el texto a minúsculas**: Para evitar que las diferencias entre mayúsculas y minúsculas interfieran en el análisis.
   - **Eliminar números**: Los números no suelen ser útiles en el análisis de texto, por lo que se eliminan.
   - **Eliminar espacios y saltos de línea adicionales**: Se asegura de que el texto esté bien formateado.
   - **Eliminar signos de puntuación**: Quita comas, puntos, signos de interrogación y otros símbolos que no aportan mucho al análisis.
   - **Tokenizar el texto**: Divide el texto en palabras individuales (llamadas "tokens").
   - **Eliminar stopwords**: Elimina palabras comunes que no aportan información relevante, como "el", "y", "pero".
   - **Etiquetar las partes del habla**: Cada palabra se marca con su tipo (sustantivo, verbo, adjetivo, etc.).
   - **Lematizar**: Convierte cada palabra a su forma base (por ejemplo, "corriendo" se convierte en "correr").

In [19]:

# Función para convertir etiquetas POS de NLTK a WordNet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Por defecto, asumimos que es sustantivo

# Función de preprocesamiento
def preprocesar_contenido(contenido):
    contenido = contenido.lower()  # Convertir a minúsculas
    contenido = re.sub(r'\d+', '', contenido)  # Eliminar números
    contenido = re.sub(r'\s+', ' ', contenido)  # Eliminar saltos de línea redundantes
    for signo in signos_puntuacion:
        contenido = contenido.replace(signo, "")  # Eliminar signos de puntuación
    contenido_tokenizado = contenido.split()  # Tokenización
    tokens_sin_stopwords = [word for word in contenido_tokenizado if word not in stopwords_set]  # Eliminar stopwords
    tokens_pos = pos_tag(tokens_sin_stopwords)  # Etiquetado POS (Part of Speech)
    tokens_lematizados = [
        lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in tokens_pos  # Lematización
    ]
    return " ".join(tokens_lematizados)  # Reconstruir el texto lematizado



* **Carga del archivo**: Se lee el archivo merged_movies_reviews.csv en un DataFrame de pandas.
* **Preprocesamiento de texto**: Se aplica la función preprocesar_contenido a la columna de críticas, limpiando el texto (minúsculas, eliminación de números, puntuación, y lematización) y guardando el resultado en una nueva columna.
* **Guardar resultado**: El DataFrame con las críticas procesadas se guarda en un nuevo archivo CSV (processed_movies_reviews.csv).

Finalmente, se imprime un mensaje confirmando que el archivo ha sido guardado.

In [20]:
# Cargar el CSV combinado
df_combined = pd.read_csv('merged_movies_reviews.csv')

# Aplicar preprocesamiento a la columna 'review_content' y guardar en una nueva columna
df_combined['processed_review_content'] = df_combined['review_content'].apply(preprocesar_contenido)

# Guardar el resultado en un nuevo archivo CSV
df_combined.to_csv('processed_movies_reviews.csv', index=False)
print("Archivo procesado guardado como 'processed_movies_reviews.csv'")

Archivo procesado guardado como 'processed_movies_reviews.csv'


# Diseño del Sistema
La función calcular_similitud_coseno calcula la similitud coseno entre una consulta de texto query y el contenido del DataFrame, con el objetivo de encontrar qué tan similar es cada texto a la consulta.

1. **Manejo de valores nulos**:  
   La columna especificada se limpia asegurando que no haya valores nulos, reemplazándolos por cadenas vacías.

2. **Preprocesamiento de la query**:  
   La consulta query se preprocesa utilizando la función preprocesar_contenido para normalizar el texto.

3. **Creación del corpus**:  
   Se combina la query preprocesada con el contenido del DataFrame. Esto forma un "corpus" que contiene todas las críticas y la consulta.

4. **Vectorización con TF-IDF**:  
   Se utiliza TfidfVectorizer para convertir el texto en vectores numéricos, donde cada vector refleja la importancia relativa de las palabras en el contexto del corpus.

5. **Cálculo de la similitud coseno**:  
   Se calcula la similitud coseno entre el vector de la consulta query_vector y los vectores de las críticas contenido_vectors.

6. **Añadir resultados al DataFrame**:  
   Los valores de similitud obtenidos se agregan como una nueva columna llamada cosine_similarity en el DataFrame original.

In [14]:

# Función para calcular la similitud coseno con la query
def calcular_similitud_coseno(df, query, column="processed_review_content"):
    # Asegurarse de que no haya valores nulos en la columna
    df[column] = df[column].fillna('')

    # Preprocesar la query
    query_procesada = preprocesar_contenido(query)
    print(f"Query procesada: {query_procesada}")  # Para verificar

    # Combinar la query con el contenido del DataFrame
    corpus = df[column].tolist() + [query_procesada]  # Último elemento será la query procesada

    # Vectorizar usando TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre la query y los textos
    query_vector = tfidf_matrix[-1]  # Última fila es el vector de la query procesada
    contenido_vectors = tfidf_matrix[:-1]  # Todos menos el último

    similitudes = cosine_similarity(query_vector, contenido_vectors).flatten()

    # Agregar la similitud como una nueva columna
    df['cosine_similarity'] = similitudes
    return df

Query procesada: great act engaging plot
                rotten_tomatoes_link  \
2111                  m/a_great_wall   
5082   m/dark_night_of_the_scarecrow   
9315               m/life_in_a_metro   
9744                  m/madame_curie   
4695                 m/color-of-lies   
17192               m/wrong_is_right   
15655              m/the_unseen_2016   
5131       m/day_of_the_flowers_2012   
16788        m/welcome-to-dongmakgol   
1244           m/1092437-in_too_deep   

                                    movie_title  \
2111                               A Great Wall   
5082                Dark Night of the Scarecrow   
9315                         Life in a... Metro   
9744                               Madame Curie   
4695   The Color of Lies (Au coeur du mensonge)   
17192                            Wrong Is Right   
15655                                The Unseen   
5131                         Day of the Flowers   
16788                     Welcome to Dongmakgol   
1244    

# Simulación de Consultas y Resultados
1. **Carga el DataFrame** con críticas de películas procesadas desde un archivo CSV.
2. **Solicita una consulta** al usuario, y si está vacía, termina el programa.
3. **Calcula la similitud coseno** entre la consulta y las críticas usando la función calcular_similitud_coseno.
4. **Ordena los resultados** por similitud, mostrando las 20 críticas más relevantes.
5. (Opcional) **Guarda los resultados** ordenados en un archivo CSV.

In [25]:
# Cargar el DataFrame procesado desde un archivo CSV
df_combined = pd.read_csv('processed_movies_reviews.csv')

while True:
    # Solicitar al usuario que ingrese la consulta
    query = input("\nIngresa la consulta para buscar (o presiona Enter para salir): ").strip()

    # Salir si la consulta está vacía
    if not query:
        print("Gracias por usar el buscador. ¡Hasta luego!")
        break

    # Calcular la similitud de coseno entre la consulta y las reseñas de películas
    df_resultado = calcular_similitud_coseno(df_combined, query)

    # Ordenar los resultados por la columna de similitud en orden descendente
    df_resultado = df_resultado.sort_values(by='cosine_similarity', ascending=False)

    # Mostrar los 20 resultados principales
    columnas_interes = [
        'rotten_tomatoes_link',
        'movie_title',
        'movie_info',
        'genres',
        'tomatometer_status',
        'cosine_similarity'
    ]
    print("\nResultados más relevantes:")
    print(df_resultado[columnas_interes].head(20))

    # Guardar los resultados en un archivo CSV
    #df_resultado.to_csv('reviews_with_similarity.csv', index=False)
    #print("\nEl archivo 'reviews_with_similarity.csv' ha sido actualizado con los resultados.")


Ingresa la consulta para buscar (o presiona Enter para salir): complex characters, emotional depth, and thought-provoking themes
Query procesada: complex character emotional depth thoughtprovoking theme

Resultados más relevantes:
                          rotten_tomatoes_link  \
3301                          m/beethovens_3rd   
2043   m/a-gai-waak-juk-jaap-project-a-part-ii   
15682                       m/the_waiting_room   
1602      m/1187621-itty_bitty_titty_committee   
16251                               m/twenty8k   
14978                      m/the_keeping_hours   
6274                      m/female_perversions   
9254                m/let_the_devil_wear_black   
13472                         m/spring_forward   
3007                  m/baader_meinhof_complex   
2218                    m/a_violent_separation   
16984                      m/whos_driving_doug   
4368                          m/changing_lanes   
15348                         m/the_price_2017   
1908              